In [1]:
from sklearn import svm 
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

In [2]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("soccerdata.sqlite")

match =  pd.read_sql_query("SELECT * FROM Match", con)
team =  pd.read_sql_query("SELECT * FROM Team", con)
team_attributes =  pd.read_sql_query("SELECT * FROM Team_Attributes", con)
player_attributes =  pd.read_sql_query("SELECT * FROM Player_Attributes", con)

#Be sure to close the connection.
con.close()

In [3]:
#match data
match = match.loc[match['league_id'] == 1729,] #filter by league
match = match[['season','date', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal',
           'home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
            "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
            "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
            "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
            "away_player_10", "away_player_11"]]
match.index = range(0, len(match["date"]))

#rename columns
match.columns = ["Season", "Date", "Home_Team_ID", "Away_Team_ID", "Home_Goals", "Away_Goals",
                'home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
                "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
                "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
                "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
                "away_player_10", "away_player_11"]

In [4]:
#what is currently present in the team member columns is the player's ID from the player attribute file.
#Let's find aggregate measures by team using the data in the player attributes df

match["HomeTeamRating"] = 0
match["AwayTeamRating"] = 0
match["HomeShortPass"] = 0
match["AwayShortPass"] = 0
match["HomeLongPass"] = 0
match["AwayLongPass"] = 0
match["HomeFreeKickAccuracy"] = 0
match["AwayFreeKickAccuracy"] = 0
match["HomeBallControl"] = 0
match["AwayBallControl"] = 0
match["HomeSprintSpeed"] = 0
match["AwaySprintSpeed"] = 0
match["HomeShotPower"] = 0
match["AwayShotPower"] = 0
match["HomeInterception"] = 0
match["AwayInterception"] = 0
match["HomePenalties"] = 0
match["AwayPenalties"] = 0
match["HomeStandingTackle"] = 0
match["AwayStandingTackle"] = 0
match["HomeSlindingTackle"] = 0
match["AwaySlindingTackle"] = 0

In [5]:
for i in range(0, len(match["Home_Goals"])):
    #get all the player IDs
    players = match.loc[i, ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
            "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
            "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
            "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
            "away_player_10", "away_player_11"]]

    for player in players[:11]: #home team
        #get the stats for the player - this gives all the stats
        player_stats = player_attributes.loc[player_attributes["player_api_id"] == player, ]    
        #now only take the rating on the day of the match
        current_stats = player_stats[player_stats.date < match.loc[i, "Date"]].sort_values(by = 'date', ascending = False)[:1]
        try:
            match.loc[i, "HomeTeamRating"] = match.loc[i, "HomeTeamRating"] + current_stats.loc[current_stats.index[0], "overall_rating"] / 11
            match.loc[i, "HomeShortPass"] = match.loc[i, "HomeShortPass"] + current_stats.loc[current_stats.index[0],"short_passing"] / 11
            match.loc[i, "HomeLongPass"] = match.loc[i, "HomeLongPass"] + current_stats.loc[current_stats.index[0],"long_passing"] / 11
            match.loc[i, "HomeFreeKickAccuracy"] = match.loc[i, "HomeFreeKickAccuracy"] + current_stats.loc[current_stats.index[0],"free_kick_accuracy"] / 11
            match.loc[i, "HomeBallControl"] = match.loc[i, "HomeBallControl"] + current_stats.loc[current_stats.index[0],"ball_control"] / 11
            match.loc[i, "HomeSprintSpeed"] = match.loc[i, "HomeSprintSpeed"] + current_stats.loc[current_stats.index[0],"sprint_speed"] / 11
            match.loc[i, "HomeShotPower"] = match.loc[i, "HomeShotPower"] + current_stats.loc[current_stats.index[0],"shot_power"] / 11
            match.loc[i, "HomeInterception"] = match.loc[i, "HomeInterception"] + current_stats.loc[current_stats.index[0],"interceptions"] / 11
            match.loc[i, "HomePenalties"] = match.loc[i, "HomePenalties"] + current_stats.loc[current_stats.index[0],"penalties"] / 11
            match.loc[i, "HomeStandingTackle"] = match.loc[i, "HomeStandingTackle"] + current_stats.loc[current_stats.index[0],"standing_tackle"] / 11
            match.loc[i, "HomeSlindingTackle"] = match.loc[i, "HomeSlindingTackle"] + current_stats.loc[current_stats.index[0],"sliding_tackle"] / 11
        except IndexError:
            pass

    for player in players[11:]: #away team
        #get the stats for the player - this gives all the stats
        player_stats = player_attributes.loc[player_attributes["player_api_id"] == player, ]    
        #now only take the rating on the day of the match
        current_stats = player_stats[player_stats.date < match.loc[i, "Date"]].sort_values(by = 'date', ascending = False)[:1]
        try:
            match.loc[i, "AwayTeamRating"] = match.loc[i, "AwayTeamRating"] + current_stats.loc[current_stats.index[0], "overall_rating"] / 11
            match.loc[i, "AwayShortPass"] = match.loc[i, "AwayShortPass"] + current_stats.loc[current_stats.index[0],"short_passing"] / 11
            match.loc[i, "AwayLongPass"] = match.loc[i, "AwayLongPass"] + current_stats.loc[current_stats.index[0],"long_passing"] / 11
            match.loc[i, "AwayFreeKickAccuracy"] = match.loc[i, "AwayFreeKickAccuracy"] + current_stats.loc[current_stats.index[0],"free_kick_accuracy"] / 11
            match.loc[i, "AwayBallControl"] = match.loc[i, "AwayBallControl"] + current_stats.loc[current_stats.index[0],"ball_control"] / 11
            match.loc[i, "AwaySprintSpeed"] = match.loc[i, "AwaySprintSpeed"] + current_stats.loc[current_stats.index[0],"sprint_speed"] / 11
            match.loc[i, "AwayShotPower"] = match.loc[i, "AwayShotPower"] + current_stats.loc[current_stats.index[0],"shot_power"] / 11
            match.loc[i, "AwayInterception"] = match.loc[i, "AwayInterception"] + current_stats.loc[current_stats.index[0],"interceptions"] / 11
            match.loc[i, "AwayPenalties"] = match.loc[i, "AwayPenalties"] + current_stats.loc[current_stats.index[0],"penalties"] / 11
            match.loc[i, "AwayStandingTackle"] = match.loc[i, "AwayStandingTackle"] + current_stats.loc[current_stats.index[0],"standing_tackle"] / 11
            match.loc[i, "AwaySlindingTackle"] = match.loc[i, "AwaySlindingTackle"] + current_stats.loc[current_stats.index[0],"sliding_tackle"] / 11
        except IndexError:
            pass

In [6]:
#team data
#extract those teams that took part in league = 1729
hometeams = match["Home_Team_ID"].unique()
awayteams = match["Away_Team_ID"].unique()
leagueTeams = np.concatenate((hometeams, awayteams), axis = 0)
team = team.loc[team['team_api_id'].isin(leagueTeams) ,] 

#get data for attendance of matches
teamStats = pd.read_csv("Attendance.csv")
teamStats.columns = ["Season", "Team", "Home Support", "Away Support", "Age"]
teamStats["Team_ID"] = 0
#add the column Away support to match
#add team id to teamStats
for i in teamStats["Team"].unique():
    try:
        id = team.loc[team['team_long_name'] == i.strip() ,"team_api_id"]
        teamStats.loc[teamStats["Team"] == i, "Team_ID"] = id[id.index[0]]
    except:
        pass
    
#merge match and teamStats
match = pd.merge(match, teamStats, how = "left", left_on = ["Season", "Home_Team_ID"], right_on = ["Season", "Team_ID"])

#delete unwanted columns from match
#match.drop('Stage', axis=1, inplace=True)
match.drop('Team_ID', axis=1, inplace=True)
match.drop('Team', axis=1, inplace=True)

In [7]:
#create a class variable for "Win", "Lose", "Tie" - WRT home team
match["Outcome"] = 0
for i in range(0, len(match["Outcome"])):
    home_goals = match.loc[i, "Home_Goals"]
    away_goals = match.loc[i, "Away_Goals"]
    if home_goals > away_goals:
        match.loc[i,'Outcome'] = 2
    elif home_goals == away_goals:
        match.loc[i,'Outcome'] = 1
    elif home_goals < away_goals:
        match.loc[i,'Outcome'] = 0

In [8]:
def getLastXMatchesResults_Home(date, team, x = 10):    
    #Filter team matches from matches
    teamMatches = match[(match['Home_Team_ID'] == team)]                           
    #Filter x last matches from team matches
    xMatches = teamMatches[teamMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
    #Calculate win percentage
    won = len(xMatches['Outcome'] == 2) / 10
    return won

def getLastXMatchesResults_Away(date, team, x = 10):    
    #Filter team matches from matches
    teamMatches = match[(match['Away_Team_ID'] == team)]                           
    #Filter x last matches from team matches
    xMatches = teamMatches[teamMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
    #Calculate win percentage
    won = len(xMatches['Outcome'] == 2) / 10
    return won
    
def getLastXMatchesResultsAgainstEachOther(date, home_team, away_team, x = 10):    
    #Find matches of both teams
    homeMatches = match[(match['Home_Team_ID'] == home_team) & (match['Away_Team_ID'] == away_team)]    
    awayMatches = match[(match['Home_Team_ID'] == away_team) & (match['Away_Team_ID'] == home_team)] 
    #Get last x matches - Home
    try:    
        xMatches = homeMatches[homeMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
        homeWin = len(xMatches['Outcome'] == 2) / 10
    except:
        xMatches = homeMatches[homeMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:homeMatches.shape[0],:]
        homeWin = 0
        #Check for error in data
        if(xMatches.shape[0] > x):
            print("Error in obtaining matches")
    
    #Get last x matches - Away
    try:    
        xMatches = awayMatches[awayMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:x,:]
        awayWin = len(xMatches['Outcome'] == 2) / 10
    except:
        xMatches = awayMatches[awayMatches.Date < date].sort_values(by = 'Date', ascending = False).iloc[0:awayMatches.shape[0],:]
        awayWin = 0
        #Check for error in data
        if(xMatches.shape[0] > x):
            print("Error in obtaining matches")
    
            
    #Return data
    return np.array([homeWin, awayWin])


In [9]:
#create columns for last X matches
match["HomeTeamWins_Home"] = 0
match["HomeTeamWins_Away"] = 0
match["AwayTeamWins_Home"] = 0
match["AwayTeamWins_Away"] = 0
match["HomeVsAway_Home"] = 0
match["HomeVsAway_Away"] = 0

for i in range(0, len(match["Season"])):
    match.loc[i, "HomeTeamWins_Home"] = getLastXMatchesResults_Home(match.loc[i, "Date"],  match.loc[i, "Home_Team_ID"])
    match.loc[i, "HomeTeamWins_Away"] = getLastXMatchesResults_Away(match.loc[i, "Date"],  match.loc[i, "Home_Team_ID"])
    match.loc[i, "AwayTeamWins_Home"] = getLastXMatchesResults_Home(match.loc[i, "Date"],  match.loc[i, "Away_Team_ID"])
    match.loc[i, "AwayTeamWins_Away"] = getLastXMatchesResults_Away(match.loc[i, "Date"],  match.loc[i, "Away_Team_ID"])
    temp =  getLastXMatchesResultsAgainstEachOther(match.loc[i, "Date"],match.loc[i, "Home_Team_ID"],match.loc[i, "Away_Team_ID"])
    match.loc[i, "HomeVsAway_Home"] = temp[0]
    match.loc[i, "HomeVsAway_Away"] = temp[1]

In [10]:
#check for missing values - filling them with 0
match.isnull().values.any()
match = match.fillna(value = 0)

In [ ]:
#make the outcome column the last one
match.insert(len(match.columns),'Outcome1', match['Outcome'])
match.drop('Outcome', axis = 1, inplace = True)
match.rename(columns = {'Outcome1':'Outcome'}, inplace = True)

In [11]:
match.to_csv("DataSetForClassification.csv")